In [12]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
from os import system
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
# for dataloader
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

from tqdm.notebook import tqdm as tqdm

### set GPU

In [13]:
# GPU setting
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

### DataLoader

In [14]:
# 將英文字母投影到 0~25，SOS、EOD分別為 26, 27
class CharDict:
    def __init__(self):
        self.word2index = {} # 單詞 --> 索引  {"0":a, "1":b, ......}
        self.index2word = {} # 索引 --> 單詞  {"a":0, "b":1, ......}
        self.n_words = 0     # 累計

        for i in range(26):
            self.addWord(chr(ord('a') + i))
        
        tokens = ["SOS", "EOS"]
        for t in tokens:
            self.addWord(t)

    def addWord(self, word):
        # 判斷單詞是否已存在，如果不存在，加個，同時統計字符出現頻率
        if word not in self.word2index:
            self.word2index[word] = self.n_words  # 單詞對應的索引
            self.index2word[self.n_words] = word  # 索引對應的單詞
            self.n_words += 1                     # 索引加一

    def longtensorFromString(self, strs):
        strs = ["SOS"] + list(strs) + ["EOS"]     # strs 極為加上SOS EOS的總字串
        return torch.LongTensor([self.word2index[chars] for chars in strs])
    
    def stringFromLongtensor(self, line, show_token=False, check_end=True):
        strs = ""
        for i in line:
            chars = self.index2word[i.item()]
            if len(chars) > 1:      # len(SOS)=len(EOS)=3, len(正常字元)=1
                if show_token:
                    __chars = "<{}>".format(ch)   # SOS, EOS
                else:
                    __chars = ""
            else:
                __chars = chars    # __ch = a,b,c,d,......
            strs += __chars        # 組在一起
            if check_end and chars == "EOS":
                break
        return strs

In [15]:
# dataloader 
class wordsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            f = './train.txt'
        else:
            f = './test.txt'
        
        with open(f) as file:
            contents = file.read()

        self.words = contents.split()   # 讀字詞
        self.n_words = len(self.words)  # 總共幾個字詞
        
        del(contents)
        
        if not train:
            self.targets = np.array([
                [0, 3], #sp -> p
                [0, 2], #sp -> pg
                [0, 1], #sp -> tp
                [0, 1], #sp -> tp
                [3, 1], #p  -> tp
                [0, 2], #sp -> pg
                [3, 0], #p  -> sp
                [2, 0], #pg -> sp
                [2, 3], #pg -> p
                [2, 1], #pg -> tp
            ])
        
        self.tenses = [
            'simple-present', 'third-person', 'present-progressive', 'simple-past'
        ]
        
        self.chardict = CharDict()
        self.train = train
        
        for i in range(len(self.words)):
            self.words[i] = self.chardict.longtensorFromString(self.words[i])
            
    def __len__(self):
        if self.train:
            return self.n_words
        else:
            return len(self.targets)
        
    def __getitem__(self, index):
        if self.train:
            condition = index % len(self.tenses)    # condition：時態 (共四種 0~3), index抓值
            return index, self.words[index], condition 
        else:   # testing
            i  = self.words[2*index]                # input字詞
            o  = self.words[2*index+1]
            condition_i = self.targets[index, 0]    # input的時態
            condition_o = self.targets[index, 1]
            
            return i, condition_i, o, condition_o

### VAE Model

In [16]:
class VAE(nn.Module):  # conditional VAE
    # Encoder
    class EncoderRNN(nn.Module):
        def __init__(self, input_size, hidden_size):
            super(VAE.EncoderRNN,self).__init__()
            self.hidden_size = hidden_size
            self.embedding   = nn.Embedding(input_size, hidden_size) # embedding成詞向量
            self.lstm         = nn.LSTM(hidden_size, hidden_size)     # useing LSTM

        def forward(self, input, hidden_state, cell_state):
            # LSTM INPUT format ： (seq_length, batch_size, embedding_dim)
            # 先把形狀為 (batch_size, seq_length) 的 input 轉置，再把每個 value (char index) 轉成 embedding vector
            embedded = self.embedding(input).view(1, 1, -1)   # (seq_length(字母), batch_size)
            output, (hidden_state, cell_state) = self.lstm(embedded, (hidden_state, cell_state))
            return output, hidden_state, cell_state
        
        # Inputs: input, (h_0, c_0), shape：(seq_len, batch, input_size)
        # Initialize ho, co  (num_layers * num_directions, batch, hidden_size)
        def init_h0(self, size): 
            return torch.zeros(1, 1, size, device=device)

        def init_c0(self):
            return torch.zeros(1, 1, self.hidden_size, device=device)

    # Decoder
    class DecoderRNN(nn.Module):
        def __init__(self, input_size, hidden_size):
            super(VAE.DecoderRNN, self).__init__()
            self.hidden_size = hidden_size
            self.embedding = nn.Embedding(input_size, hidden_size)
            self.lstm = nn.LSTM(hidden_size, hidden_size)
            self.out = nn.Linear(hidden_size, input_size)
            
        
        # output即 predict結果
        def forward(self, input_, hidden_state, cell_state): 
            
            output = self.embedding(input_).view(1, 1, -1)
            output = F.relu(output)
            output, (hidden_state, cell_state) = self.lstm(output, (hidden_state, cell_state))
            output = self.out(output[0])

            return output, hidden_state, cell_state

        def init_h0(self):
            pass

        def init_c0(self):
            return torch.zeros(1, 1, self.hidden_size, device=device)

    def __init__(self, input_size, hidden_size, latent_size, conditional_size, max_length):
        super(VAE, self).__init__()
        self.encoder     = self.EncoderRNN(input_size, hidden_size)
        self.decoder     = self.DecoderRNN(input_size, hidden_size)
        self.input_size  = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.max_length  = max_length
        self.conditional_size = conditional_size
        
        self.hidden2mean   = nn.Linear(hidden_size, latent_size)
        self.hidden2logvar = nn.Linear(hidden_size, latent_size)
        
        latentcondition2hidden_size = latent_size + conditional_size
        self.latentcondition2hidden = nn.Linear(latentcondition2hidden_size, hidden_size)

    def forward(self, input_tensor, encoder_hidden_state, encoder_cell_state, c, use_teacher_forcing):
        # encoder forwarding
        input_length = input_tensor.size(0)        # string長度
        
        for en_idx in range(input_length-2):    # 一個個單詞餵進encoder，扣掉SOS, EOS
            _, encoder_hidden_state, encoder_cell_state = self.encoder(
                    input_tensor[en_idx+1], encoder_hidden_state, encoder_cell_state
                )
        
        # middle 
        mean   = self.hidden2mean(encoder_hidden_state)
        logvar = self.hidden2logvar(encoder_hidden_state)
        
        # sampling a point
        # 相當於採樣過程提前，取樣一個點，通過reparameterize使得整個網路forward過程中的函數都可導，之後進行BP
        latent = self.reparameterize(mean, logvar)
        
        decoder_hidden_state = self.latentcondition2hidden(torch.cat((latent, c), dim=-1))
        decoder_cell_state   = self.decoder.init_c0()
        decoder_input = input_tensor[0] #傳字母進來，初始--> SOS
        
        # decoder forwarding

        predict_distribution = torch.zeros(input_length, self.input_size, device=device) # input length ：字母長度+SOS, EOS
                                                                                         # input_size： 28
        predict_output = None
               
        for de_idx in range(input_length-1):  #輸入觀點不看EOS，從SOS開始丟； ENCODE DECODE的CONDITION相同，所以長度相同

            output, decoder_hidden_state, decoder_cell_state = self.decoder(
                                        decoder_input, decoder_hidden_state, decoder_cell_state
                                    )
            
            predict_distribution[de_idx] = output[0] # 一堆機率(各種class：0~27種)
            predict_class = output.topk(1)[1] # 看是哪個class，取最大的那個的index --> 可以再轉換成字母 
            
            if predict_output is not None:
                predict_output = torch.cat((predict_output, predict_class))
            else:
                predict_output = predict_class
            
             # use teacher forcing
            if use_teacher_forcing : 
                decoder_input = input_tensor[de_idx + 1]  # 餵進label
                
            else:
                if predict_class.item() == EOS_token:  
                    break
                decoder_input = predict_class

        return predict_output, predict_distribution, mean, logvar

    def inference(self, input_tensor, encoder_hidden_state, encoder_cell_state, c):
        # encoder     
        input_length = input_tensor.size(0)      # string長度
        for en_idx in range(input_length-2):    # 一個個單詞餵進encoder，扣掉SOS, EOS
            _, encoder_hidden_state, encoder_cell_state = self.encoder(
                    input_tensor[en_idx+1], encoder_hidden_state, encoder_cell_state
                )
        # middle        
        mean=self.hidden2mean(encoder_hidden_state)
        logvar=self.hidden2logvar(encoder_hidden_state)
        
        latent=self.reparameterize(mean,logvar)
        
        decoder_hidden_state = self.latentcondition2hidden(torch.cat((latent, c), dim=-1))
        decoder_cell_state = self.decoder.init_c0()
        decoder_input = input_tensor[0]

        # decoder 
        predict_output = None

        for di in range(self.max_length): 
            output, decoder_hidden_state, decoder_cell_state = self.decoder(decoder_input,decoder_hidden_state,
                                                                            decoder_cell_state)
            predict_class = output.topk(1)[1]
            predict_output = torch.cat((predict_output, predict_class)) if predict_output is not None else predict_class

            if predict_class.item() == EOS_token:
                break
            decoder_input = predict_class
        

        return predict_output

    def generate(self,latent,label_tensor):
        label_tensor = torch.LongTensor([label_tensor]) # 時態的label
        label_tensor = torch.zeros(1, 4).scatter_(1, label_tensor.unsqueeze(0), 1) #to one hot
        label_tensor = label_tensor.view(1, 1, -1).to(device)
 
        decoder_hidden_state = self.latentcondition2hidden(torch.cat((latent, label_tensor), dim=-1))
        decoder_cell_state=self.decoder.init_c0()
        decoder_input = torch.tensor([[SOS_token]], device=device)

        # decoder 
        
        predict_output = None
        for de_idx in range(self.max_length): 
            output, decoder_hidden_state, decoder_cell_state = self.decoder(decoder_input, 
                                                            decoder_hidden_state, decoder_cell_state
                                                                           )
            predict_class  = output.topk(1)[1]
            
            if predict_output is not None:
                predict_output = torch.cat((predict_output, predict_class))
            else:
                predict_output = predict_class
#             predict_output = torch.cat((predict_output, predict_class)) if predict_output is not None else predict_class

            if predict_class.item() == EOS_token:
                break
            decoder_input = predict_class

        return predict_output

    
    # reparameterization trick
    def reparameterize(self, mean, logvar):
        std    = torch.exp(0.5*logvar)
        z_star = torch.randn_like(std)  # 隨機採樣一個點 z*
        latent = mean + z_star*std      # z = z*＊exp(logvar/2) + mean
        return latent


### Train/Test Func declare

In [17]:
def loss_function(predict_distribution, predict_output_length, target, mu, logvar):
    criterion = nn.CrossEntropyLoss()
    CEloss = criterion(predict_distribution[:predict_output_length], target[1:predict_output_length+1]) 
    #如果字母原本7個, 前面ditribution 0~7, length =8, 

    # KL(N(mu,variance)||N(0,1))
    KLloss = -0.5 * torch.sum(1 + logvar - mu**2 - logvar.exp())
    return CEloss, KLloss

def train(vae, loader_train, learning_rate, teacher_forcing_ratio, kl_weight, tensor2string):
    vae.train()
    optimizer = optim.SGD(vae.parameters(), lr = learning_rate) 
    
    total_CEloss = 0
    total_KLloss = 0
#     total_BLEUscore = 0
    
    for idx, input_tensor, label_tensor in (loader_train):
        optimizer.zero_grad()
        input_tensor = input_tensor[0] # batch size = 1，這邊是取一個string
        
        label_tensor = torch.zeros(1, 4).scatter_(1, label_tensor.unsqueeze(0), 1) # to one hot
        label_tensor = label_tensor.view(1, 1, -1)
        
        input_tensor = input_tensor.to(device)
        label_tensor = label_tensor.to(device)

        # init hidden_state
        h0_size = vae.hidden_size - vae.conditional_size
        h0 = vae.encoder.init_h0(h0_size)
        
        encoder_hidden_state = torch.cat((h0, label_tensor), dim=-1)
        
        # init cell_state
        encoder_cell_state = vae.encoder.init_c0()

        # forwarding one word by calling VAE Model forwarding
        use_teacher_forcing = True if random.random()< teacher_forcing_ratio else False
        
        predict_output, predict_distribution, mean, logvar = vae(input_tensor, encoder_hidden_state, encoder_cell_state,
                                                            label_tensor, use_teacher_forcing)
        
        CEloss, KLloss = loss_function(predict_distribution, predict_output.size(0),   # output string (第0維) 長度
                                      input_tensor.view(-1), mean, logvar)
        loss = CEloss + kl_weight * KLloss
        total_CEloss += CEloss.item()
        total_KLloss += KLloss.item()

        predict = tensor2string(predict_output)
        target  = tensor2string(input_tensor)
        
        #update
        loss.backward()
        optimizer.step()
        
    return vae, total_CEloss/len(loader_train), total_KLloss/len(loader_train)

def evaluate(vae,loader_test,tensor2string):
    vae.eval()
    result = []
    total_BLEUscore = 0
    with torch.no_grad():
        for in_tensor, in_label, out_tensor, out_label in loader_test:
            in_tensor  = in_tensor[0].to(device)
            out_tensor = out_tensor[0].to(device)
#             in_tensor, out_tensor = in_tensor[0].to(device),out_tensor[0].to(device)
            
            in_label  = torch.zeros(1, 4).scatter_(1, in_label.unsqueeze(0), 1) #to one hot
            in_label  = in_label.view(1, 1, -1)
            out_label = torch.zeros(1, 4).scatter_(1, out_label.unsqueeze(0), 1) #to one hot
            out_label = out_label.view(1, 1, -1)
            
            in_label  = in_label.to(device)
            out_label = out_label.to(device)

            # init hidden_state
            h0_size = vae.hidden_size - vae.conditional_size
            h0 = vae.encoder.init_h0(h0_size)
            encoder_hidden_state = torch.cat((h0, in_label), dim=-1)
            
            # init cell_state
            encoder_cell_state = vae.encoder.init_c0()

            # forwarding one word by calling VAE Mode inference
            predict_output = vae.inference(in_tensor, encoder_hidden_state, encoder_cell_state, out_label)
            
            target_word = tensor2string(out_tensor)
            predict_word= tensor2string(predict_output)
            
            result.append([tensor2string(in_tensor), target_word, predict_word])
            total_BLEUscore += compute_bleu(predict_word, target_word)
            
            print('--------------------------')
            print(' input word  :{}      '.format((tensor2string(in_tensor))))
            print(' target word :{}      '.format(target_word))
            print(' predict word:{}      '.format(predict_word))
        
    return total_BLEUscore/len(loader_test)

def generateWord(cvae, latent_size, tensor2string):
    cvae.eval()
    result=[]
    with torch.no_grad():
        for i in range(100):
            latent = torch.randn(1, 1, latent_size).to(device)
            tmp = []
            for tense in range(4):
                word = tensor2string(cvae.generate(latent, tense))
                tmp.append(word)

            result.append(tmp)
    return result

In [18]:
#compute BLEU-4 score
def compute_bleu(output, reference):
    cc = SmoothingFunction()
    if len(reference) == 3:
        weights = (0.33,0.33,0.33)
    else:
        weights = (0.25,0.25,0.25,0.25)
    return sentence_bleu([reference], output,weights=weights,smoothing_function=cc.method1)

def Gaussian_score(words):
    words_list = []
    score = 0
    yourpath = './train.txt' 
    with open(yourpath,'r') as fp:
        for line in fp:
            word = line.split(' ')
            word[3] = word[3].strip('\n')
            words_list.extend([word])
        for t in words:
            for i in words_list:
                if t == i:
                    score += 1
    return score/len(words)

def get_teacher_forcing_ratio(epoch):
    teacher_forcing_ratio = 1.-(0.0018*epoch)
    
    return teacher_forcing_ratio
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_kl_weight(epoch, epochs, kl_annealing_type):

    assert kl_annealing_type=='monotonic' or kl_annealing_type=='cycle','kl_annealing_type not exist!'
    if kl_annealing_type == 'monotonic':
        if epoch < 50:
            return 0.02*epoch
        else:
            return 1
    else: #cycle
        period = epochs//2 
        epoch %= period
        if epoch < 249:
            return sigmoid((epoch-125)/25)/2
        else:
            return 0


### Paramter settin'

In [19]:
SOS_token = 26
EOS_token = 27

#Parameter of VAE
input_size = 28
hidden_size = 256  # LSTM hidden size
latent_size = 32
conditional_size = 4
MAX_LENGTH = 17

#Parameter of training
LR = 0.05
kl_annealing_type = 'cycle'
EPOCHS = 500

### Load data

In [20]:
train_dataset = wordsDataset(True)
train_loader = torch.utils.data.DataLoader(
                            train_dataset,
                            batch_size=1,
                            shuffle=True,
                            num_workers=2,
                        )


test_dataset = wordsDataset(False)
test_loader = torch.utils.data.DataLoader(
                            test_dataset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=2,
                        )

### Load Model

In [21]:
vae = VAE(input_size, hidden_size, latent_size, conditional_size, MAX_LENGTH).to(device)

In [22]:
CEloss = []
KLloss = []
teacher_ratio = []
KL_weight = []
BLEU_score = []
gaussian_score = []
Best_BLEU = 0.0

    # training
for epoch in tqdm(range(EPOCHS)):
    teacher_forcing_ratio = get_teacher_forcing_ratio(epoch+1)
    kl_weight = get_kl_weight(epoch+1, EPOCHS, kl_annealing_type)
    teacher_ratio.append(teacher_forcing_ratio)
    KL_weight.append(kl_weight)
    
    #training
    vae, CEloss_temp, KLloss_temp = train(vae, train_loader, LR, teacher_forcing_ratio,
                                      kl_weight, train_dataset.chardict.stringFromLongtensor)
    
    CEloss.append(CEloss_temp)
    KLloss.append(KLloss_temp)
    
    #testing
    BLEUscore = evaluate(vae, test_loader, train_dataset.chardict.stringFromLongtensor)
    BLEU_score.append(BLEUscore)
    
    if (BLEUscore > Best_BLEU):
        torch.save({'state_dict': vae.state_dict()}, './model_weight/model_VAE_Best_BLEU_submit_final.pth.tar')
        Best_BLEU = BLEUscore

    save_path = './model_weight/every_epoch/model_VAE_epoch_' + str(epoch) + '_submit_final.pth.tar'
    torch.save({'state_dict': vae.state_dict()}, save_path)
    
    print ('-------------Train Epoch : no.{} -------------'.format(epoch))
    print ('teacher ratio : {}  || KL weight : {}'.format(teacher_forcing_ratio, kl_weight))
    print ('CE Loss = {}'.format(CEloss_temp))
    print ('KL Loss = {}'.format(KLloss_temp))
    print ('BLEU score in this epoch = {} \n'.format(BLEUscore))

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:surger      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:surger      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:chunted      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:surger      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:chunted      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:chunted      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:chunted      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:surger      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:surger      
------

-------------Train Epoch : no.6 -------------
teacher ratio : 0.9874  || KL weight : 0.004418200230303861
CE Loss = 1.1688438530437522
KL Loss = 12.924034089554397
BLEU score in this epoch = 0.12293093620999082 

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:abounded      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:abet      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:bested      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:expended      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:steers      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:spilling      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:flailed      
------------------------

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:abounded      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:abet      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:bings      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:experiences      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:snetches      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:slumping      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:flare      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:function      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:functioned    

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:abandoned      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:abetting      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:begins      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:expends      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:sneeps      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:spilling      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:flare      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:furnish      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:functioned     

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:abandoned      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:abetting      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:begs      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:expends      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:senses      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:sliming      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:flare      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:function      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:functioned      


...

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:seseeee      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:seaiiiigg      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:seaesss      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:seaesss      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:seaesss      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:seaiiiigg      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:seae      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:seae      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:seseeee      
---

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:seseeedd      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:seaiiinn      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:sesesss      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:sesesss      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:sesesss      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:seaiiinn      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:seae      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:seae      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:seseeedd      
---

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:seaeeee      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:seaiiiin      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:seaeess      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:seaeess      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:seaeess      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:seaiiiin      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:seaee      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:seaee      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:seaeeee      
---

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:seaeeee      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:seaiiiig      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:seassss      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:seassss      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:seassss      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:seaiiiig      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:seae      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:seae      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:seaeeee      
-----

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:seseeeed      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:sesiiin      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:sesesss      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:sesesss      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:sesesss      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:sesiiin      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:sesee      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:sesee      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:seseeeed      
---

### Plot result

In [23]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))

plt.plot(BLEU_score, color='b', linewidth=2)
plt.plot(CEloss)
plt.plot(KLloss)
plt.plot(KL_weight, color='r', linestyle = "-" , linewidth=3)
plt.plot(teacher_ratio,  color='g', linewidth = 4 )

plt.title('CVAE with cycle teacher ratio_submit_final')
plt.ylabel('value')
plt.xlabel('epoch')

plt.legend(['BLEU score', 'CE Loss',
            'KL loss', 'KL Weight',
            'teacher ratio'], loc=0)
plt.grid(True)
plt.show()
plt.savefig('CVAE with cycle teacher ratio_submit_final.png')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### Load model and count BLEU

In [26]:
checkpoint = torch.load('./model_weight/model_VAE_Best_BLEU_submit_final.pth.tar')
vae.load_state_dict(checkpoint['state_dict'])

BLEUscore = evaluate(vae,test_loader,train_dataset.chardict.stringFromLongtensor)
print('\nBLEU score : ', BLEUscore)

--------------------------
 input word  :abandon      
 target word :abandoned      
 predict word:abandoned      
--------------------------
 input word  :abet      
 target word :abetting      
 predict word:abetting      
--------------------------
 input word  :begin      
 target word :begins      
 predict word:begs      
--------------------------
 input word  :expend      
 target word :expends      
 predict word:expends      
--------------------------
 input word  :sent      
 target word :sends      
 predict word:steens      
--------------------------
 input word  :split      
 target word :splitting      
 predict word:splitting      
--------------------------
 input word  :flared      
 target word :flare      
 predict word:flare      
--------------------------
 input word  :functioning      
 target word :function      
 predict word:function      
--------------------------
 input word  :functioning      
 target word :functioned      
 predict word:functioned     

### Gaussian score

In [54]:
generated_words = generateWord(vae, latent_size, train_dataset.chardict.stringFromLongtensor)
gaussianscore = Gaussian_score(generated_words)
print(generated_words)
print("-------------------------------")
print('Gaussian score：', gaussianscore)

[['straighten', 'stares', 'startling', 'stared'], ['repair', 'meases', 'preseaning', 'misspended'], ['restort', 'interests', 'restoring', 'restorting'], ['applie', 'applies', 'applieing', 'applied'], ['rolake', 'rolakes', 'rangling', 'roladed'], ['enablort', 'enabots', 'expenting', 'expended'], ['disunite', 'disunites', 'disuniting', 'disunited'], ['happen', 'happeing', 'faying', 'happed'], ['idaliate', 'idaliates', 'salating', 'salated'], ['fighten', 'fightens', 'fightening', 'fightened'], ['rolate', 'rolats', 'rolating', 'rolated'], ['assail', 'wits', 'assailing', 'assailed'], ['telt', 'telts', 'telting', 'told'], ['injure', 'ducks', 'ducying', 'introduced'], ['moving', 'moving', 'moving', 'moving'], ['sulked', 'sulkeels', 'sulking', 'sulked'], ['giess', 'giesses', 'giessing', 'giessed'], ['blowe', 'blowers', 'blowing', 'blowed'], ['renect', 'renects', 'renecting', 'renected'], ['smack', 'smacks', 'smacking', 'smacked'], ['announce', 'announces', 'announcing', 'announced'], ['satisfy